# Use our crops and weather data to build a predictive model

In [13]:
# load all needed libraries

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.svm import SVC
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score,explained_variance_score,mean_absolute_error
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_columns', None)

In [2]:
# import data

folder = 'C:/Users/szums/AIBootCampPrime/AgProject3/Resources/'

#crops data
cropsdf = pd.read_csv(folder + 'cropsANNUALNCwide.csv',delimiter=',') 
display(cropsdf.info)

print(f'{len(cropsdf)} rows read from crops file.')

#weather data
wxdf = pd.read_excel(folder + 'MergedWeatherDataAnnual.xlsx')

print(f'{len(wxdf)} rows read from weather file.')


<bound method DataFrame.info of      YEAR  BARLEY_ACRES   BARLEY_$  BARLEY_BU    BARLEY_BU_ACRE  CORN_ACRES  \
0    1866           NaN        NaN          NaN             NaN         NaN   
1    1867           NaN        NaN          NaN             NaN         NaN   
2    1868           NaN        NaN          NaN             NaN         NaN   
3    1869           NaN        NaN          NaN             NaN         NaN   
4    1870           NaN        NaN          NaN             NaN         NaN   
..    ...           ...        ...          ...             ...         ...   
153  2019       11000.0  1287000.0     396000.0            66.0    990000.0   
154  2020       15000.0  2079000.0     693000.0            77.0    990000.0   
155  2021       13000.0  1979000.0     455000.0            65.0    960000.0   
156  2022       16000.0  4023000.0     759000.0            69.0    820000.0   
157  2023       16000.0  3344000.0     760000.0            76.0    950000.0   

          CORN_$  C

158 rows read from crops file.
45 rows read from weather file.


In [3]:
# we have weather data from 2000 - 2023 and we have crops data from 1866 to 2023.  make the years in each file match up.

wxdf1 = wxdf.dropna()

wxminyear = wxdf1['year'].min()
wxmaxyear = wxdf1['year'].max()
print('min year in wx file',wxminyear,'max year',wxmaxyear)

crminyear = cropsdf['YEAR'].min()
crmaxyear = cropsdf['YEAR'].max()
print('min year in crop file',crminyear,'max year',crmaxyear)

cropsdf1 = cropsdf[(cropsdf['YEAR'] >= wxminyear) & (cropsdf['YEAR'] <= wxmaxyear)]

if len(wxdf1) == len(cropsdf1):
    print('Matching rows')
else:
    print('ERROR: wxdf1 and cropsdf1 have different numbers of years')

display('cropsdf1',cropsdf1.shape)
display('wxdf1',wxdf1.shape)


min year in wx file 2000 max year 2020
min year in crop file 1866 max year 2023
Matching rows


'cropsdf1'

(21, 49)

'wxdf1'

(21, 14)

In [4]:
# make sure the dataframes are sorted by year

wx_sorted = wxdf1.sort_values(by=['year'])

crops_sorted = cropsdf1.sort_values(by=['YEAR'])

wx_sorted.columns

Index(['year', 'Fall (SON)', 'Spring (MAM)', 'Summer (JJA)', 'Winter (DJF)',
       'Annual Avg', 'precp Annual avg', 'Precip Fall', 'Precip Spring',
       'Precip Summer', 'Precip Winter', 'Weeks of Severe drought',
       'Weeks of Extreme Drought', 'Weeks of Exceptional Drought'],
      dtype='object')

In [10]:
# now work with target values one by one

# targets = ['BARLEY_BU_ACRE','CORN_BU_ACRE','COTTON_LB_ACRE','HAY_T_ACRE','OATS_BU_ACRE','PEANUTS_LB_ACRE','PEPPERS, BELL_CWT_ACRE',\
#            'SOYBEANS_BU_ACRE','SQUASH_CWT_ACRE','SWEET_CWT_ACRE','TOBACCO_LB_ACRE','WHEAT_BU_ACRE']

X = wx_sorted.drop(['year','Annual Avg','precp Annual avg'],axis='columns')
X.columns

#columns_to_scale = ['']
crops_sorted.dropna(axis=1, how='any',inplace=True)
targets = crops_sorted.columns.drop('YEAR')

targets

Index(['BARLEY_ACRES', 'BARLEY_$', 'BARLEY_BU  ', 'BARLEY_BU_ACRE',
       'CORN_ACRES', 'CORN_$', 'CORN_BU  ', 'CORN_BU_ACRE', 'COTTON_ACRES',
       'COTTON_LB', 'COTTON_LB_ACRE', 'HAY_ACRES', 'HAY_$', 'HAY_T   ',
       'HAY_T_ACRE', 'OATS_ACRES', 'OATS_$', 'OATS_BU  ', 'OATS_BU_ACRE',
       'PEANUTS_ACRES', 'PEANUTS_$', 'PEANUTS_LB', 'PEANUTS_LB_ACRE',
       'PEPPERS, BELL_ACRES', 'PEPPERS, BELL_$', 'PEPPERS, BELL_CWT',
       'PEPPERS, BELL_CWT_ACRE', 'SOYBEANS_ACRES', 'SOYBEANS_$',
       'SOYBEANS_BU  ', 'SOYBEANS_BU_ACRE', 'SQUASH_ACRES', 'SQUASH_$',
       'SQUASH_CWT', 'SQUASH_CWT_ACRE', 'SWEET_ACRES', 'SWEET_$', 'SWEET_CWT',
       'SWEET_CWT_ACRE', 'TOBACCO_ACRES', 'TOBACCO_$', 'TOBACCO_LB',
       'TOBACCO_LB_ACRE', 'WHEAT_ACRES', 'WHEAT_$', 'WHEAT_BU  ',
       'WHEAT_BU_ACRE'],
      dtype='object')

In [14]:

mse_list = []
r2_list = []
model_list = []
target_list = []
y_pred_list = []
y_test_list = []
ev_list = []
mae_list = []

models = [LinearRegression,SVR,DecisionTreeRegressor,RandomForestRegressor,GradientBoostingRegressor]

for t in targets:
    print('Working on',t)
    y = crops_sorted[t]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.fit_transform(X_test)

    scaler_y = StandardScaler()
    y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
    y_test_scaled = scaler_y.fit_transform(y_test.values.reshape(-1, 1)).flatten()


    for m in models:
        print('...',m)
        target_list.append(t)

        this_model = m()
        this_model.fit(X_train_scaled,y_train_scaled)
        y_pred = this_model.predict(X_test_scaled)
        y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

        y_pred_list.append(y_pred_original)
        y_test_list.append(y_test)
        model_list.append(this_model)

        mse = mean_squared_error(y_test_scaled,y_pred)
        mse_list.append(mse)

        r2 = r2_score(y_test_scaled, y_pred)
        r2_list.append(r2)

        mae = mean_absolute_error(y_test_scaled,y_pred)
        mae_list.append(mae)

        ev_score = explained_variance_score(y_test_scaled,y_pred)
        ev_list.append(ev_score)

        #print('\tMean square error:',mse)
        #print('\tR2 score',r2)

resultsdf = pd.DataFrame({'Target':target_list,
                          "model" : model_list,
                          'MSE':mse_list,
                          'R2': r2_list,
                          'Mean_abs_error': mae_list,
                          'Explained var' : ev_list,
                          'y_pred': y_pred_list,
                          'y_test': y_test_list})
    
resultsdf.to_excel(folder+'cropWeatherModelResults1.xlsx', index=False)
print('fininshed')


Working on BARLEY_ACRES
... <class 'sklearn.linear_model._base.LinearRegression'>
... <class 'sklearn.svm._classes.SVR'>
... <class 'sklearn.tree._classes.DecisionTreeRegressor'>
... <class 'sklearn.ensemble._forest.RandomForestRegressor'>
... <class 'sklearn.ensemble._gb.GradientBoostingRegressor'>
Working on BARLEY_$
... <class 'sklearn.linear_model._base.LinearRegression'>
... <class 'sklearn.svm._classes.SVR'>
... <class 'sklearn.tree._classes.DecisionTreeRegressor'>
... <class 'sklearn.ensemble._forest.RandomForestRegressor'>
... <class 'sklearn.ensemble._gb.GradientBoostingRegressor'>
Working on BARLEY_BU  
... <class 'sklearn.linear_model._base.LinearRegression'>
... <class 'sklearn.svm._classes.SVR'>
... <class 'sklearn.tree._classes.DecisionTreeRegressor'>
... <class 'sklearn.ensemble._forest.RandomForestRegressor'>
... <class 'sklearn.ensemble._gb.GradientBoostingRegressor'>
Working on BARLEY_BU_ACRE
... <class 'sklearn.linear_model._base.LinearRegression'>
... <class 'sklear

# Model Performance summary

1. The GradientBoostingRegressor and DecisionTreeRegressor performed better than the others according to all 4 measures of accuracy
2. The models worked better with certain crops.  Hay